## 01. Import Libraries

In [2]:
# import libraries:

import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

## 02. Import Data:

In [5]:
# importing our most recent data set:

df = pd.read_csv('/Users/piperdutcher/Documents/Data-Visualizations/Bike-Sharing/Data/chronological_w_ride_counts.csv', index_col = 0)

/var/folders/02/_ydtglj578xfqwjs1hm49rhc0000gn/T/ipykernel_4997/831939132.py:3: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/piperdutcher/Documents/Data-Visualizations/Bike-Sharing/Data/chronological_w_ride_counts.csv', index_col = 0)


In [7]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,average_temp,trip_length_minutes,daily_ride_count
0,3369D016821ACD0A,classic_bike,2022-01-01 09:55:25.013,2022-01-01 10:05:47.093,Lafayette St & E 8 St,5788.13,8 Ave & W 16 St,6072.11,40.730207,-73.991026,40.740983,-74.001702,casual,2022-01-01,11.6,10.368000,5105
1,0617BD2A38E149CF,electric_bike,2022-01-01 15:10:00.630,2022-01-01 15:18:29.885,Frederick Douglass Blvd & W 117 St,7688.12,Park Ave & E 124 St,7682.01,40.805159,-73.954692,40.804555,-73.939686,casual,2022-01-01,11.6,8.487583,5105
2,040D9CEB9A11C469,electric_bike,2022-01-01 02:35:34.345,2022-01-01 02:58:21.676,W 42 St & 6 Ave,6517.08,W 100 St & Broadway,7580.01,40.754920,-73.984550,40.797372,-73.970412,member,2022-01-01,11.6,22.788850,5105
3,7E01F3FA7BAC8F02,classic_bike,2022-01-01 15:29:24.789,2022-01-01 15:40:11.693,1 Ave & E 16 St,5779.08,Hudson St & W 13 St,6115.06,40.732219,-73.981656,40.740057,-74.005274,member,2022-01-01,11.6,10.781733,5105
4,5FBAD09F5F32B2E4,electric_bike,2022-01-01 08:38:24.623,2022-01-01 08:48:09.812,W 59 St & 10 Ave,7023.04,West End Ave & W 94 St,7524.09,40.770513,-73.988038,40.794165,-73.974124,member,2022-01-01,11.6,9.753150,5105


In [9]:
df['trip_length_minutes'].value_counts()

trip_length_minutes
4.664800     22
5.247417     22
3.759783     22
5.628167     22
5.269300     22
             ..
38.210117     1
50.982900     1
18.023250     1
33.203867     1
34.156983     1
Name: count, Length: 2152864, dtype: int64

In [11]:
df.shape

(7441508, 17)

## 03. Analysis

In [14]:
# creating a new dataframe to see which ride paths are the most popular.
# first step is to create the new dataframe, then group by starting and ending stations
# finally, creating a new count column with a value of one to allow us to begin the count :)


df['path_count'] = 1
df_rides = df.groupby(['start_station_name', 'end_station_name'])['path_count'].count().reset_index()

df_rides

,start_station_name,end_station_name,path_count
0,1 Ave & E 110 St,1 Ave & E 110 St,201
1,1 Ave & E 110 St,1 Ave & E 18 St,1
2,1 Ave & E 110 St,1 Ave & E 30 St,2
3,1 Ave & E 110 St,1 Ave & E 44 St,5
4,1 Ave & E 110 St,1 Ave & E 62 St,7
...,...,...,...
664511,Yankee Ferry Terminal,Vesey Pl & River Terrace,1
664512,Yankee Ferry Terminal,Water St & Fletcher St,3
664513,Yankee Ferry Terminal,West St & Chambers St,2
664514,Yankee Ferry Terminal,West St & Liberty St,2


In [16]:
# exporting this for future use!

df_rides.to_csv('/Users/piperdutcher/Documents/Data-Visualizations/Bike-Sharing/Data/most_popular_ride_paths.csv')

In [18]:
# checking to make sure the sum of all path_count is equivalent to the above df size (891137) to ensure all rides have been tracked!

print(df_rides['path_count'].sum())

7441508


In [20]:
df_rides['path_count'].describe()

count    664516.000000
mean         11.198388
std          30.051404
min           1.000000
25%           1.000000
50%           3.000000
75%           9.000000
max        2991.000000
Name: path_count, dtype: float64

In [22]:
# looks good! we can proceed with creating our map of most popular rides!